<a href="https://colab.research.google.com/github/nyquist/aoe2de/blob/main/AOE2DE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Config Replay

In [1]:
#game_id = "358345344" #"357732557"

Get latest match ids

In [2]:
import requests
#profile_id = "18446743"
relic_id = "222475"
#match_type = 4
#game = "age2"

#profile_url = f"https://www.ageofempires.com/stats/?profileId={profile_id}&game={game}&matchType={match_type}"
profile_url = f"https://www.aoe2insights.com/user/{relic_id}/matches/"
response = requests.get(profile_url)
#print (response.status_code)
#print (response.headers)
import re
matches = re.finditer(r'href="/match/(\d+)/"', response.text)
games = []
for match in matches:
    found_game_id = match.group(1)
    games.append(found_game_id)
game_id = games[0]
print (game_id)



358345344


#Environment Setup

In [3]:
! [[ -d "aoe2de" ]] && (cd aoe2de && git pull) || git clone https://github.com/nyquist/aoe2de.git
! ls -la ./aoe2de/utils
! tail aoe2de/mtmExtractBuildingsfromJson.py

Cloning into 'aoe2de'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 82 (delta 45), reused 13 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (82/82), 2.02 MiB | 2.93 MiB/s, done.
Resolving deltas: 100% (45/45), done.
total 12
drwxr-xr-x 2 root root 4096 Dec 15 00:00 .
drwxr-xr-x 4 root root 4096 Dec 15 00:00 ..
-rw-r--r-- 1 root root   22 Dec 15 00:00 test.py
        else:
            mtable = self.ecoTable
        mtable.set_style(MARKDOWN)
        return mtable.get_string()
      

if __name__ == '__main__':
    print ("Need to fix passing file_name. Try using it as a module")
    analyzer = BuildingsAnalyzer("AgeIIDE_Replay_358345344.aoe2record.json")
    print (analyzer.getTable("eco"))


In [4]:
import os

def list_files_with_dynamic_sizes(path="."):
    """Lists all files and their sizes with dynamic units (B, kB, MB, etc.) in the given directory.

    Args:
        path (str, optional): The directory path to list files from. Defaults to "." (current directory).
    """
    for entry in os.scandir(path):
        if entry.is_file():
            file_size = os.path.getsize(entry.path)
            unit_index = 0
            units = ["B", "kB", "MB", "GB", "TB"]
            while file_size >= 1024 and unit_index < len(units) - 1:
                file_size /= 1024
                unit_index += 1
            print(f"{entry.name}: {file_size:.2f} {units[unit_index]}")

# Call the function to list files and sizes with dynamic units in the current directory


# Download record

In [5]:
import requests

replay_url = f"https://aoe.ms/replay/?gameId={game_id}&profileId=18446743"  # Replace with the actual URL you copied

response = requests.get(replay_url)
response.raise_for_status()  # Raise an exception for bad responses
print (response.status_code)
content_disposition = response.headers['Content-Disposition']
import re
match = re.search(r'filename="?([^";]+)"?', content_disposition)
if match:
    file_name = match.group(1)
    print(f"Extracted filename: {file_name}")
    with open(file_name, "wb") as f:
      f.write(response.content)
else:
    print("Filename not found in Content-Disposition header.")


print(f"Replay file downloaded as {file_name}")
list_files_with_dynamic_sizes()





200
Extracted filename: AgeIIDE_Replay_358345344.zip
Replay file downloaded as AgeIIDE_Replay_358345344.zip
AgeIIDE_Replay_358345344.zip: 1.48 MB


### Unzip record file

In [6]:
zip_file_name = file_name
record_file_name = file_name.replace(".zip", ".aoe2record")

import zipfile

try:
    with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
        zip_ref.extractall(".")  # Extract all files to the current directory
    print(f"Successfully extracted files from {zip_file_name}")

    # Find the extracted replay file (assuming it's an .mgz file)
    extracted_file_name = None
    for filename in os.listdir("."):
        print (filename)
        if filename == record_file_name:
            extracted_file_name = filename
            break

    if extracted_file_name:
        # Process the extracted replay file
        print ("Extracted file:", extracted_file_name)
    else:
        print("Error: Could not find extracted replay file (.mgz).")

except zipfile.BadZipFile:
    print(f"Error: {zip_file_name} is not a valid zip file.")
except FileNotFoundError:
    print(f"Error: {zip_file_name} not found.")
except RuntimeError as e:
    if "could not parse" in str(e):
        print(f"Error parsing the replay file: {e}")
        print("The replay file might be corrupted or in an unsupported format.")
        print("Please try re-downloading the replay file or using a different one.")
    else:
        raise  # Re-raise the exception if it's not related to parsing

Successfully extracted files from AgeIIDE_Replay_358345344.zip
.config
AgeIIDE_Replay_358345344.aoe2record
Extracted file: AgeIIDE_Replay_358345344.aoe2record


## Convert file to json

In [7]:
import json
! pip install mgz
from mgz.model import parse_match, serialize
f = extracted_file_name
outFile=f"{extracted_file_name}.json"
print (f)
with open(f, 'rb') as h:
    match = parse_match(h)
    # print(json.dumps(serialize(match), indent=2))
    json_object=json.dumps(serialize(match), indent=2)
    with open(outFile, "w") as o:
        o.write(json_object)

list_files_with_dynamic_sizes()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mgz: filename=mgz-1.8.28-py3-none-any.whl size=72422 sha256=291a7fe6dcae32cdc534e611de16f3effcb2643f5a9e721cd620997e34202771
  Stored in directory: /root/.cache/pip/wheels/71/f0/68/23ed8196623ac82c61348a1fdee66446ac501bf9a1bf635720
  Created wheel for construct: filename=construct-2.8.16-py2.py3-none-any.whl size=70258 sha256=fd0b867e87cdf0d3bee5fbd74a12d69acc192250ee413f088c909be18c78a3a8
  Stored in directory: /root/.cache/pip/wheels/78/37/f7/3a9d487ec99b26560f68e030c537774c3ed4e4e39128c40bf7
  Created wheel for aocref: filename=aocref-2.0.20-py3-none-any.whl size=94222 sha256=5979da970bc9582722dd9b5d2041738

# Analysis

In [8]:
with open(outFile, 'r') as json_file:
    json_data = json.load(json_file)

## Buildings

In [9]:
! tail aoe2de/mtmExtractBuildingsfromJson.py
! pip3 install prettytable

from aoe2de.mtmExtractBuildingsfromJson import BuildingsAnalyzer

all_buildings = BuildingsAnalyzer(outFile)

        else:
            mtable = self.ecoTable
        mtable.set_style(MARKDOWN)
        return mtable.get_string()
      

if __name__ == '__main__':
    print ("Need to fix passing file_name. Try using it as a module")
    analyzer = BuildingsAnalyzer("AgeIIDE_Replay_358345344.aoe2record.json")
    print (analyzer.getTable("eco"))
email_to_mihai 1
nyquist 3
gabisan 5
risk 7
CANAFARCI 2
衝浪小王子 4
sHmOoPie :) 6
球球你别跑 8
['email_to_mihai', 'CANAFARCI', 'nyquist', '衝浪小王子', 'gabisan', 'sHmOoPie :)', 'risk', '球球你别跑']
[{}, {}, {}, {}, {}, {}, {}, {}]
+----------------+----------------+-----------+---------+------------+---------+-------------+------+------------+
|    Cladire     | email_to_mihai | CANAFARCI | nyquist | 衝浪小王子 | gabisan | sHmOoPie :) | risk | 球球你别跑 |
+----------------+----------------+-----------+---------+------------+---------+-------------+------+------------+
|    Barracks    |       1        |     17    |    8    |     14     |    2    |      10     |  8   |     1      

In [14]:
def discordMessage(content):
  from google.colab import userdata
  userdata.get('webhook')

  import requests

  webhook_url = userdata.get('webhook')
  data = {"content": content}

  response = requests.post(webhook_url, json=data)

  if response.status_code == 204:
      print("Message sent successfully!")
  else:
      print(f"Failed to send message: {response.status_code}")

discordMessage(f"Last game {game_id}")
discordMessage(f"```{all_buildings.getTable('eco')}```")
discordMessage(f"```{all_buildings.getTable('mil')}```")
#type(all_buildings)



Message sent successfully!
Message sent successfully!
Message sent successfully!


## Army